<a href="https://colab.research.google.com/github/prateekrana17/SPARK-NLP-RAG-LLM/blob/main/Persona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
major_version, minor_version= torch.cuda.get_device_capability()


print(f"CUDA major version: {major_version}")
print(f"CUDA minor version: {minor_version}")

CUDA major version: 7
CUDA minor version: 5


In [3]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ueli_cur/unsloth_c04bfd076f9a48d0a82ae94cd4ef5f76
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ueli_cur/unsloth_c04bfd076f9a48d0a82ae94cd4ef5f76
  Resolved https://github.com/unslothai/unsloth.git to commit 64bb8cfd512a9dcd860d21563b624676f7432ec5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
if major_version>=8:
  !pip install --no-deps packaging ninja einops flasth-attn xformers trl peft accelerate bitsandbytes
else:
  !pip install --no-deps xformers trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.7 MB/s eta 0:00:00


In [5]:
from unsloth import FastLanguageModel

import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [6]:
max_seq_len = 2048
dtype= None
load_in_4bit = True

In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
'''alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

'''


'alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{}\n\n### Input:\n{}\n\n### Response:\n{}"""\n\nEOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN\ndef formatting_prompts_func(examples):\n    instructions = examples["instruction"]\n    inputs       = examples["input"]\n    outputs      = examples["output"]\n    texts = []\n    for instruction, input, output in zip(instructions, inputs, outputs):\n        # Must add EOS_TOKEN, otherwise your generation will go on forever!\n        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN\n        texts.append(text)\n    return { "text" : texts, }\npass\n\nfrom datasets import load_dataset\ndataset = load_dataset("", split = "train")\ndataset = dataset.map(formatting_prompts_func, batched = True,)\n\n'

In [8]:
import os
import re
import unicodedata
from collections import Counter
import json
import requests
import random

In [9]:
# Read and preprocess the text file for writing style
file_path = 'Barrie_Painter 1.txt'

with open(file_path, 'r') as file:
    text = file.read()

# Normalize text
text = text.lower()
text = unicodedata.normalize('NFKD', text)

# Remove special characters, numbers, and punctuation


# Tokenize text
words = text.split()
word_counts = Counter(words)

# Calculate sentence lengths
sentences = text.split('.')
sentence_lengths = [len(sentence.split()) for sentence in sentences]

# Extract vocabulary
vocabulary = set(words)

# Extract most common words (for vocabulary and style)
common_words = word_counts.most_common(100)
common_words = [word for word, _ in common_words]



# save LLM extracted Writing & Personality
personality = {
    "Name": "Barrie Painter",
    "Designation": "Chief Sustainability Officer",
    "Personaliy & Writing Style Description" : "Conclusion Barrie Painter exemplifies a strategic, visionary leader with a deep commitment to sustainability and inclusivity. His leadership style is characterized by strategic thinking, innovation, and a collaborative approach, making him a pivotal figure in driving Motherson’s sustainability and marketing initiatives. His writing style complements his leadership, being clear, engaging, inspirational, and inclusive, effectively communicating his vision and strategies to a diverse audience. Through his leadership and communication, Barrie consistently demonstrates his dedication to creating impactful solutions for the organization and the broader community. Writing Style Review for Barrie Painter. Clear and Engaging.Barrie’s writing style is clear, direct, and engaging. He articulates complex ideas in a straightforward manner, making his communications accessible to a broad audience. This clarity is particularly important in his dual roles, where he must convey intricate sustainability strategies and marketing initiatives effectively.Inspirational and MotivationalBarrie’s writings often carry an inspirational and motivational tone. Phrases like “Together with our collective efforts, we can make a significant difference” and “Let us commit to conserving resources wherever possible” are designed to inspire and galvanize his audience. His use of motivational language fosters a sense of shared purpose and commitment among his readers.Strategic and Informative.Barrie’s writing is strategic and informative, providing detailed insights into the organization’s goals, initiatives, and progress. His updates on the sustainability strategy, DEI training, and Vision 2025 objectives are not only informative but also reflect strategic planning and foresight. This style ensures that stakeholders are well-informed about the organization’s direction and the rationale behind various initiatives.Inclusive and Collaborative.His writing style emphasizes inclusivity and collaboration. By frequently using inclusive language such as “we” and “our,” Barrie fosters a sense of unity and collective responsibility. His references to team achievements and group discussions highlight his collaborative approach and respect for team contributions.Use of Modern Communication Tools.Barrie effectively utilizes modern communication tools and social media to amplify his message. The use of hashtags like #sustainability, #DEI, and #proudtobepartofplanetearth reflects his understanding of digital communication trends and his ability to engage with a global audience. This approach not only broadens the reach of his messages but also aligns with contemporary communication practices.Leadership and Vision.Barrie Painter exhibits a strong, visionary leadership style characterized by a deep commitment to sustainability and comprehensive strategic thinking. His extensive experience in senior leadership roles has equipped him with a robust understanding of the automotive industry and the broader market dynamics. Barrie’s transition from Executive Vice President, Global Sales and Marketing, to Chief Sustainability Officer and Head of Global Marketing and Communications demonstrates his adaptability and capability to lead in diverse functional areas. His vision is not just limited to short-term goals but extends to long-term, sustainable growth for the organization, emphasizing the importance of Environmental, Social, and Governance (ESG) principles.Strategic Thinker and Innovator.Barrie’s career reflects a strong propensity for strategic thinking and innovation. His ability to spearhead sales, marketing, business development, and product innovation initiatives showcases his forward-thinking mindset and innovative approach. He understands the critical intersection between marketing, communications, and sustainability, leveraging this understanding to create impactful solutions that benefit both the business and the planet. His strategic objectives review for Vision 2025 and beyond illustrates his dedication to setting and achieving ambitious, forward-looking goals.Commitment to Sustainability and DEI.Barrie’s role as Chief Sustainability Officer involves not only developing and leading the organization’s sustainability strategy but also championing Diversity, Equity, and Inclusion (DEI). His active participation in DEI training and discussions demonstrates his commitment to fostering an inclusive workplace culture. Barrie’s approach to sustainability is holistic, encompassing environmental stewardship, social responsibility, and robust governance practices. His mantra, “Turn Off, Turn Down,” underscores his dedication to reducing resource consumption and minimizing waste, reflecting his personal and professional commitment to sustainability.Collaborative and People-Oriented. Barrie places significant importance on collaboration and people-oriented leadership. His participation in the Global Sustainability Summit and interactions with the global HR leaders indicate his belief in the power of collective effort and expertise. He values the contributions of his team and stakeholders, fostering an environment where diverse perspectives are encouraged and respected. This collaborative approach not only enhances organizational cohesion but also drives sustainable success.Communication Skill.Barrie excels in communication, both in articulating complex strategies and in engaging with diverse stakeholders. His ability to convey the importance of sustainability and DEI in an accessible and compelling manner is evident in his communications. His use of social media hashtags and engagement with broader audiences demonstrates his proficiency in modern communication tools and platforms. Barrie’s communication style is clear, motivating, and inclusive, effectively driving his message across various channels."
    }

    # Save extracted features
features = {
    "common_words": common_words,
    "sentence_lengths": sentence_lengths,
    "vocabulary": list(vocabulary), "persona": personality
}

with open("barrie_painter_features.json", "w") as f:
    json.dump(features, f)


In [10]:
# LLM generated JSON prompts from User Data
import json
responses= [
  {
    "prompt": "What did the Motherson Sustainability Development Team recently do to promote diversity, equity, and inclusion (DEI) in the workplace?",
    "response": "The people of Motherson are extremely important and therefore understanding and nurturing Diversity, equity, and inclusion (DEI) is essential. In view of this our Sustainability Development Team received DEI training from Cleo at Mission Diverse. The session increased our self-awareness around the topic and led to group discussions on the importance of DEI in the workplace. As a topic that is continuously evolving, we need to remain informed in order that Motherson can benefit from this and meet the expectations of our stakeholders, which of course includes our employees together with our HR partners across Motherson we will continue to progress diversity, equity and inclusion. #sustainability #sustainabilityatmotherson #DiversityEquityInclusion #DEI #proudtobepartofplanetearth"
  },
  {
    "prompt": "What did Motherson's Group Sustainability Development Team recently accomplish, and what are they looking forward to in the future?",
    "response": "Motherson\u2019s Group Sustainability Development Team completed this week a strategic objectives review for the remainder of Vision 2025 and beyond. In depth reviews and lively discussions on the most important topics across the scope of ESG or for Motherson Planet People & Governance. Thank you to the team and watch this space for more details and follow-up actions. Looking forward to our annual review on the progress of the actions coming from the 2023 Sustainability Summit. #sustainabilityatmotherson #proudtobepartofplanetearth"
  },
  {
    "prompt": "What is the \"Turn Off, Turn Down\" mantra created at Motherson's Global Sustainability Summit 2023 and what is its purpose?",
    "response": "Created at Motherson's first-ever Global Sustainability Summit 2023, this powerful mantra guides us in reducing consumption and minimising resource waste across our businesses. By embracing \"Turn Off, Turn Down\" in our daily work routines, we become champions of optimum utilisation of available resources and contribute to the overall success of our organisation.\nLet us commit to conserving resources wherever possible, reducing our environmental footprint, reducing costs, and promoting a sustainable future. Each day let us reflect and ask ourselves what steps have we taken to conserve resources today?\nTogether with our collective efforts, we can make a significant difference."
  },
  {
    "prompt": "What is the author looking forward to next week, and why is it important to them?",
    "response": "Really looking forward to joining our global HR leaders at their conference meeting next week in Portugal. \u2018People\u2019 sustainability is very important to Motherson and there is no better place to explore this further than being with our experts.\n#togetherwemakeithappen"
  },
  {
    "prompt": "Create a welcome message for the Mothersonone sustainability page that encourages followers to share ideas and initiatives.",
    "response": "Welcome to sustainability on Mothersonone. Follow this space to get regular updates and insights on all things sustainability at Motherson. The central sustainability development team are here to help and we look forward to you sharing ideas and initiatives from wherever you are in the Motherson world.\n\nSustainability - #theresponsibilitylieswithusall."
  },
  {
    "prompt": "Watch the video below to hear from our Chief Sustainability Officer, Barrie Painter",
    "response": "You might be wondering what's on the horizon for the sustainability team at Motherson.\n\nWatch the video below to hear from our Chief Sustainability Officer, Barrie Painter about our upcoming double materiality assessment and how you can help.\n\nhttps://motherson.workvivo.com/spaces/52502/pages/dual-materiality\n\n#doublematerialitysurvey"
  }
]

with open("barrie_painter_examples.json", "w") as f:
    json.dump(responses,f)

In [11]:
# Load the JSON file
json_file_path = 'barrie_painter_features.json'
with open(json_file_path, 'r') as file:
    features = json.load(file)

common_words = features['common_words']
sentence_lengths = features['sentence_lengths']
vocabulary = features['vocabulary']
persona=features['persona']


In [15]:
!pip install trl
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset ='barrie_painter_features.json',
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


TypeError: string indices must be integers

In [21]:
prompt = "What is development?"
generated_text = generate_text(prompt, model_name, pseudo_dataset, max_tokens=500)
print("Generated text:", generated_text)


NameError: name 'model_name' is not defined